In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [12]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X,Y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: {(100*correct):>1f},    Avg loss: {test_loss:>8f}")

In [17]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.166941 [    0/60000]
loss:2.163209 [ 6400/60000]
loss:2.104471 [12800/60000]
loss:2.118520 [19200/60000]
loss:2.091724 [25600/60000]
loss:2.026406 [32000/60000]
loss:2.048659 [38400/60000]
loss:1.982409 [44800/60000]
loss:1.975036 [51200/60000]
loss:1.911530 [57600/60000]
Test Error: 56.930000,    Avg loss: 1.906119
Epoch 2
-------------------------------
loss:1.937581 [    0/60000]
loss:1.917636 [ 6400/60000]
loss:1.798180 [12800/60000]
loss:1.833367 [19200/60000]
loss:1.746111 [25600/60000]
loss:1.691301 [32000/60000]
loss:1.702061 [38400/60000]
loss:1.607539 [44800/60000]
loss:1.621953 [51200/60000]
loss:1.519218 [57600/60000]
Test Error: 60.040000,    Avg loss: 1.531610
Epoch 3
-------------------------------
loss:1.596157 [    0/60000]
loss:1.567044 [ 6400/60000]
loss:1.409188 [12800/60000]
loss:1.484764 [19200/60000]
loss:1.372550 [25600/60000]
loss:1.365375 [32000/60000]
loss:1.371844 [38400/60000]
loss:1.295011 [44800/60000]
loss:1